In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [2]:
# configure g_key
gmaps.configure(api_key=g_key)


In [3]:
# read pdf from weatherpy export
df = pd.read_csv("fileOne.csv")
df.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ancud,6,CL,1580095293,87,-41.87,-73.82,50.00,2.24
1,acapulco,20,MX,1580094543,74,16.86,-99.89,80.60,8.05
2,georgetown,20,MY,1580094742,62,5.41,100.34,86.00,3.36
3,hofn,87,IS,1580094704,64,64.25,-15.21,39.74,23.09
4,sorland,100,NO,1580094796,97,67.67,12.69,28.49,28.74
5,geraldton,0,AU,1580094599,34,-28.77,114.60,95.00,8.05
6,hilo,75,US,1580094633,65,19.73,-155.09,78.80,11.41
7,cape town,20,ZA,1580094561,87,-33.93,18.42,60.01,5.82
8,college,100,US,1580094751,74,64.86,-147.80,-14.01,2.39
9,lerwick,20,GB,1580095026,60,60.15,-1.15,41.00,9.17


In [4]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]
#store humidity
humidity = df["Humidity"]



In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,max_intensity=20,point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
#Max temp 55-80
#Humidity <30%
#Cloudiness <20%
#Wind Speed <15mph
#create narrowed data frame based on specific weather conditions

hotel_df = df[df["Max Temp"]>=55]
hotel_df = hotel_df[hotel_df["Max Temp"]<=80]
hotel_df = hotel_df[hotel_df["Humidity"]<=30]
hotel_df = hotel_df[hotel_df["Cloudiness"]<=20]
hotel_df = hotel_df[hotel_df["Wind Speed"]<=15]
#create hotel name column
hotel_df["Hotel Name"]=""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
146,ankpa,0,NG,1580095315,16,7.37,7.63,68.65,4.25,
306,atar,0,MR,1580095340,28,20.52,-13.05,65.19,8.66,
314,asosa,10,ET,1580095341,19,10.07,34.53,63.57,7.96,
407,sabha,1,LY,1580095356,21,27.04,14.43,56.88,6.82,
474,kontagora,20,NG,1580094830,19,10.40,5.47,66.02,7.00,
511,biltine,0,TD,1580095373,17,14.53,20.92,59.81,14.18,
532,tahoua,0,NE,1580095378,11,14.89,5.27,64.40,6.93,
553,kousseri,0,CM,1580095381,21,12.08,15.03,60.80,8.05,


In [7]:
hotel_names =[]
hotel_lat =[]
hotel_lng =[]
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat long df
    lat = row["Lat"]
    lng = row["Lng"]

    # add location to params dict
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    
    response = requests.get(base_url, params=params)
    response = response.json()
    results = response["results"]
    try:
        hotel_names.append(results[0]["name"])
        hotel_lat.append(results[0]['geometry']["location"]['lat'])
        hotel_lng.append(results[0]['geometry']["location"]['lng'])
    except (KeyError,IndexError):
        hotel_names.append("N/A")
        hotel_lat.append("N/A")
        hotel_lng.append("N/A")


In [8]:
#insert data into hotel data frame  based upon lists in cell above

hotel_df["Hotel Name"]=hotel_names
hotel_df["Hotel Lat"]=hotel_lat
hotel_df["Hotel Lng"]=hotel_lng
hotel_df= hotel_df[hotel_df["Hotel Name"]!="N/A"]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
146,ankpa,0,NG,1580095315,16,7.37,7.63,68.65,4.25,Holiday Inn,7.12129,7.65233
306,atar,0,MR,1580095340,28,20.52,-13.05,65.19,8.66,les toiles maures,20.5419,-13.0559
314,asosa,10,ET,1580095341,19,10.07,34.53,63.57,7.96,Bamboo Paradise Hotel,10.0598,34.5447
407,sabha,1,LY,1580095356,21,27.04,14.43,56.88,6.82,Al Jabal Hotel,26.9451,14.4689
474,kontagora,20,NG,1580094830,19,10.40,5.47,66.02,7.00,Had Resources Hotel,10.403,5.473
532,tahoua,0,NE,1580095378,11,14.89,5.27,64.40,6.93,HOTEL TARKA,14.8837,5.29813
553,kousseri,0,CM,1580095381,21,12.08,15.03,60.80,8.05,"Radisson Blu Hotel, N'Djamena",12.0889,15.0712


In [9]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# print figure with makers and maker info boxes
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))